# Solving the Travelling Salesman Problem using Genetic Algorithm

The *Traveling Salesman Problem* (TSP) is a classic and extremely important optimisation problem in the field of operations research and computer science. It can be stated as follows: *Given a set of cities and the distances between each pair of cities, the goal is to find the shortest possible route that visits each city exactly once and returns back to the starting city.* The TSP is known to be a NP-hard problem, hence it doesn't have a known polynomial-time algorithm which solves it exactly. In this first lab session, we will try to leverage *Genetic Algorithm* (GA) to come up with a solution.

Let's start by importing some useful modules.

In [ ]:
import random
import numpy as np

We define a `City` class to handle our cities more easily. 

In [ ]:
class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return "(" + str(self.x) + ", " + str(self.y) + ")"
    
    def __eq__(self, other_city):
        return isinstance(other_city, City) and self.x == other_city.x and self.y == other_city.y
    
    def __ne__(self, other_city):
        return not self == other_city
    
    def distance(self, city):
        x_diff = abs(self.x - city.x)
        y_diff = abs(self.y - city.y)
        return np.sqrt((x_diff ** 2) + (y_diff ** 2))

Now we want a function to initialize our population. Remember that each route must contain each city **exactly once**.

In [ ]:
def get_cities(n_cities, x_range, y_range):
    cities = []
    for _ in range(n_cities):
        cities.append(City(x=int(random.random() * x_range), y=int(random.random() * y_range)))
    return cities

def init_population(cities, pop_size):
    routes = []
    for _ in range(pop_size):
        rem_cities = list(cities)
        route = []
        while len(rem_cities) > 0:
            route.append(rem_cities.pop(random.randint(0, len(rem_cities) - 1)))
        routes.append(route)
    return routes

We define the **fitness** as the inverse of the route distance. Since we want to minimise the distance, we want to maximise our fitness score.

In [ ]:
def fitness_score(route):
    fitness = 0
    for k in range(len(route)-1):
        fitness += route[k].distance(route[k+1])
    return fitness

We define now a function to perform **tournament selection** (or any other selection strategy you may prefer).

In [ ]:
def tournament_selection(pop, fit, k):
    tournament = []
    for _ in range(k):
        tournament.append(pop[random.randint(0, len(pop) - 1)])
    return max(tournament, key=lambda x: fit(x))

Since each individual is a permutation of a set of different cities we must ensure that crossover and mutation generate valid individuals. As for the crossover, we can choose between the **partially mapped crossover** (**PMX**) and the **cycle crossover**. Let's implement both.

In [ ]:
def pmx(parent1, parent2):
    l = len(parent1)
    child = [None] * l
    # SELECTION AND COPY OF THE FIRST RANGE OF VALUES FROM P1
    i, j = sorted(random.sample(range(l), 2))
    child[i:j+1] = parent1[i:j+1]
    # PERFORM THE PARTIAL MAPPING
    for g in range(i, j+1):
        n = parent1[g]
        m = parent2[g]
        if m not in child:
            k = parent2.index(n)
            while child[k] is not None:
                k = parent2.index(parent1[k])
            child[k] = m
    # COPY OF THE REMAINING VALUES OF P2
    for g in range(l):
        if child[g] is None:
            child[g] = parent2[g]
    return child

In [ ]:
def cx(parent1, parent2):
    l = len(parent1)
    start = random.randint(0, l-1)
    child = [None] * l
    curr = parent1.index(parent2[start])
    # FIND THE CYCLE ELEMENTS OF P1
    while curr != start:
        child[curr] = parent1[curr]
        curr = parent1.index(parent2[curr])
    child[curr] = parent1[curr]
    # COPY OF THE REMAINING VALUES OF P2
    for g in range(l):
        if child[g] is None:
            child[g] = parent2[g]
    return child

Let's check if we did everything right with simple lists of integers

In [ ]:
parent1 = [7, 2, 3, 1, 5, 4, 6]
parent2 = [2, 4, 5, 6, 1, 7, 3]


print(pmx(parent1, parent2))
print(cx(parent1, parent2))

Also the mutation needs to generate valid individuals. Choose a strategy which suits our problem.

In [ ]:
def mutate(individual, p_m):
    if random.random() < p_m:
        return individual
    i1 = random.randint(0, len(individual) - 1)
    i2 = i1
    while i1 == i2:
        i2 = random.randint(0, len(individual) - 1)
    swp = individual[i1]
    individual[i1] = individual[i2]
    individual[i2] = swp
    return individual

Now we define the function `generation`, which performs:
- Selection
- Crossover
- Mutation

One should have the possibility to include elitism.

In [ ]:
def get_elites(pop, fit, k_el):
    if k_el == 0:
        return []
    elites = list(pop)
    elites.sort(key=lambda x: fit(x))
    return elites[0:k_el]

def generation(pop, fit, crossover, p_m, t_size, elitism=True, k_el=1):
    # CODE HERE
    N = len(pop)
    selected_pop = []
    for _ in range(N - k_el):
        selected_pop.append(tournament_selection(pop, fit, t_size))
    selected_pop.extend(get_elites(pop, fit, k_el))
    xover_pop = []
    for _ in range(N):
        xover_pop.append(crossover(random.choice(selected_pop), random.choice(selected_pop)))
    return list(mutate(i, p_m) for i in xover_pop)

A GA performs a generational cycle a predefined number of times.

In [ ]:
def get_best(pop, fit):
    return max([(fit(x), x) for x in pop])

In [ ]:
def GA(cities, pop_size, n, fit, crossover, t_size = 10, n_gen = 200):
    pop = init_population(cities, pop_size)
    history = []
    for g in range(n_gen):
        pop = generation(pop, fit, crossover, 0.5, t_size)
        history.append([get_best(pop, fit)[0], g])
    return get_best(pop, fit), history

Run the Algorithm with different crossover startegies and parametrisations. For each experiment, make a line plot showing the best fitness score for each generation.

In [ ]:
cities = get_cities(25, 200, 200)

In [ ]:
res, history = GA(cities=cities, pop_size=40, n=len(cities), fit=fitness_score, crossover=pmx)
best_fit, best_route = res

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history)
plt.ylabel('Fitness')
plt.xlabel('Generation')
plt.show()

Let's plot the solution found by the GA. 

In [ ]:
route = res[1]

x = [city.x for city in route]
y = [city.y for city in route]

plt.scatter(x, y, label='Cities', color='blue')

for i in range(len(cities) - 1):
    plt.plot([x[i], x[i + 1]], [y[i], y[i + 1]], 'r-')

plt.plot([x[len(cities)-1], x[0]], [y[len(cities)-1], y[0]], 'r-')
plt.title('Solution')

# Show legend
plt.legend()

# Show the plot
plt.show()